In [89]:
import urlparse
import requests
import os
import sys
import json

*New example and endpoint from Abraham*
```
>>> import requests
>>> endpoint = "http://wes.ucsc-cgp-dev.org:8080/ga4gh/wes/v1"
>>> post_run_workflow = "/workflows"
>>> descriptor = "/home/ubuntu/mock_wes/workflow-service/testdata/md5sum.cwl"
>>> workflow_type ="CWL"
>>> workflow_type_version="v1.0"  
>>> params = {'output_file': {'path': '/outdir/md5sum.txt', 'class': 'File'}, 'input_file': {'path': '../../mock_wes/workflow-service/testdata/md5sum.input', 'class': 'File'}}
>>> body = {"workflow_url":descriptor, "workflow_params" : params, "workflow_type": workflow_type, "workflow_type_version": workflow_type_version}
>>> print requests.post(endpoint+post_run_workflow, json=body).json()
```

In [74]:
endpoint = 'http://wes.ucsc-cgp-dev.org:8080/ga4gh/wes/v1'
host = 'wes.ucsc-cgp-dev.org:8080'
proto = 'http'
post_run_workflow = "/workflows"

descriptor = '/home/ubuntu/mock_wes/workflow-service/testdata/md5sum.cwl'
workflow_type = 'CWL'
workflow_type_version = 'v1.0'  
params = {'output_file': {'path': '/outdir/md5sum.txt', 'class': 'File'}, 'input_file': {'path': '../../mock_wes/workflow-service/testdata/md5sum.input', 'class': 'File'}}
body = {"workflow_url": descriptor, "workflow_params" : params, "workflow_type": workflow_type, "workflow_type_version": workflow_type_version}
print(requests.post(endpoint+post_run_workflow, json=body).json())

{u'workflow_id': u'3c691565636c4d10b25431e873bec837'}


In [77]:
get_workflow_run(wes, '3c691565636c4d10b25431e873bec837')

{u'outputs': {u'output_file': {u'basename': u'md5sum.txt',
   u'checksum': u'sha1$5cd16de143136d95a0307bc1db27d88b57b033e9',
   u'class': u'File',
   u'format': u'http://edamontology.org/data_3671',
   u'location': u'file:///home/ubuntu/workflows/3c691565636c4d10b25431e873bec837/outdir/md5sum.txt',
   u'path': u'/home/ubuntu/workflows/3c691565636c4d10b25431e873bec837/outdir/md5sum.txt',
   u'size': 33}},
 u'request': {u'workflow_params': {u'input_file': {u'class': u'File',
    u'path': u'../../mock_wes/workflow-service/testdata/md5sum.input'},
   u'output_file': {u'class': u'File', u'path': u'/outdir/md5sum.txt'}},
  u'workflow_type': u'CWL',
  u'workflow_type_version': u'v1.0',
  u'workflow_url': u'/home/ubuntu/mock_wes/workflow-service/testdata/md5sum.cwl'},
 u'state': u'Complete',
 u'task_logs': [],
 u'workflow_id': u'3c691565636c4d10b25431e873bec837',
 u'workflow_log': {u'cmd': [u''],
  u'endTime': u'',
  u'exitCode': 0,
  u'startTime': u'',
  u'stderr': u"/home/ubuntu/wes_env/bin/

In [80]:
from bravado.client import SwaggerClient
from bravado.requests_client import RequestsClient

In [84]:
http_client = RequestsClient()
split = urlparse.urlsplit("%s://%s/" % (proto, host))

# http_client.set_api_key(
#     split.hostname, args.auth,
#     param_name='Authorization', param_in='header')
client = SwaggerClient.from_url(
    "%s://%s/swagger.json" % (proto, host),
    http_client=http_client, config={'use_models': False})

In [90]:
l = client.WorkflowExecutionService.ListWorkflows()
json.dump(l.result(), sys.stdout, indent=4)

{
    "next_page_token": "", 
    "workflows": [
        {
            "state": "Complete", 
            "workflow_id": "6ab6a9e2728b4068bca19855071a2155"
        }, 
        {
            "state": "Error", 
            "workflow_id": "8e398fb0b4d043a2a3b534c1b063fd57"
        }, 
        {
            "state": "Running", 
            "workflow_id": "7efd9cde924345768423225faf934d58"
        }, 
        {
            "state": "Complete", 
            "workflow_id": "3c691565636c4d10b25431e873bec837"
        }, 
        {
            "state": "Error", 
            "workflow_id": "8db78ed7cd324912a8af65d83f1716b6"
        }, 
        {
            "state": "Complete", 
            "workflow_id": "9002c8e40f67404c9ffbeab5f6428992"
        }, 
        {
            "state": "Error", 
            "workflow_id": "cc9e9abf668342658b893ebea793576d"
        }
    ]
}

In [104]:
l = client.WorkflowExecutionService.GetServiceInfo()
l.result

<bound method HttpFuture.result of <bravado.http_future.HttpFuture object at 0x104f2b4d0>>

## Configuration

Get endpoint and authentication info from environment variables (**note:** provided by Abraham via email).

In [23]:
# proto = os.environ.get('WES_API_PROTO')
# host = os.environ.get('WES_API_HOST')
# auth = os.environ.get('WES_API_AUTH')

# new public WES endpoint using the `wes-service` repo
host = 'wes.ucsc-cgp-dev.org:8080'
proto = 'http'

Create a simple object to store session/connection info.

In [24]:
class WorkflowServiceSession(object):
    """
    API session with workflow execution service endpoint.
    """
    def __init__(self, host, auth=None, proto='http'):
        self.endpoint = '{}://{}/ga4gh/wes/v1'.format(proto, host)
        self.headers = headers = {'Authorization': auth}


In [25]:
wes = WorkflowServiceSession(host=host, proto=proto)

## Getting info from WES

Create a generic `GET` request function (to minimize repeated code).

In [26]:
def wes_get_request(wes, target):
    """
    Generic workflow execution service API 'GET' request.
    """
    res = requests.get(
        '{}{}'.format(wes.endpoint, target),
        headers=wes.headers
    )
    # TODO: add some exception handling for different responses
    return res.json()

Use `get_info` to retrieve summary information about the WES endpoint.

In [27]:
def get_info(wes):
    """
    Collect properties of a workflow execution service server endpoint.
    """
    target = '/service-info'
    return wes_get_request(wes, target)

In [28]:
get_info(wes)

{u'engine_versions': u'cwl-runner',
 u'key_values': {},
 u'supported_filesystem_protocols': [u'file'],
 u'supported_wes_versions': u'0.1.0',
 u'system_state_counts': {},
 u'workflow_type_versions': {u'CWL': [u'v1.0']}}

Use `list_workflow_runs` to list all workflow instances/runs/jobs (completed, initialized, or running) that have been recorded in the history of the WES endpoint.

In [29]:
def list_workflow_runs(wes, page_size=100, page_token=0):
    """
    List all workflow runs in workflow execution service server endpoint.
    """
    target = '/workflows'
    return wes_get_request(wes, target)

In [30]:
list_workflow_runs(wes)

{u'next_page_token': u'',
 u'workflows': [{u'state': u'Complete',
   u'workflow_id': u'6ab6a9e2728b4068bca19855071a2155'},
  {u'state': u'Error', u'workflow_id': u'8e398fb0b4d043a2a3b534c1b063fd57'},
  {u'state': u'Running', u'workflow_id': u'7efd9cde924345768423225faf934d58'},
  {u'state': u'Error', u'workflow_id': u'8db78ed7cd324912a8af65d83f1716b6'},
  {u'state': u'Complete', u'workflow_id': u'9002c8e40f67404c9ffbeab5f6428992'},
  {u'state': u'Error', u'workflow_id': u'cc9e9abf668342658b893ebea793576d'}]}

> **Note:** `workflow_id`s are now UUIDs, not just integers; shouldn't be a problem, but can't assume that workflow runs are ordered

Use `get_workflow_run` to retrieve details about a specific workflow run by its ID.

In [19]:
def get_workflow_run(wes, workflow_run_id):
    target = '/workflows/{}'.format(workflow_run_id)
    return wes_get_request(wes, target)

In [20]:
# get details of the most recent workflow run
get_workflow_run(wes, list_workflow_runs(wes)['workflows'][-1]['workflow_id'])

{u'outputs': {},
 u'request': {u'workflow_params': {u'input_file': {u'class': u'File',
    u'path': u'../../workflow-service/testdata/md5sum.input'},
   u'output_file': {u'class': u'File', u'path': u'/outdir/md5sum.txt'}},
  u'workflow_type': u'CWL',
  u'workflow_type_version': u'v1.0',
  u'workflow_url': u'/home/ubuntu/mock_wes/workflow-service/testdata/md5sum.cwl'},
 u'state': u'Error',
 u'task_logs': [],
 u'workflow_id': u'cc9e9abf668342658b893ebea793576d',
 u'workflow_log': {u'cmd': [u''],
  u'endTime': u'',
  u'exitCode': 1,
  u'startTime': u'',
  u'stderr': u'/home/ubuntu/wes_env/bin/cwl-runner 1.0.20180501200546\nResolved \'/home/ubuntu/mock_wes/workflow-service/testdata/md5sum.cwl\' to \'file:///home/ubuntu/mock_wes/workflow-service/testdata/md5sum.cwl\'\n[workflow md5sum.cwl] start\n[step md5sum] start\nUnexpected exception\nTraceback (most recent call last):\n  File "/home/ubuntu/wes_env/local/lib/python2.7/site-packages/cwltool/workflow.py", line 646, in job\n    **kwargs):\

Use `get_workflow_run_status` to retrieve the status of a workflow run (this is the same as the `state` property of a workflow run object). 

In [21]:
def get_workflow_run_status(wes, workflow_run_id):
    target = '/workflows/{}/status'.format(workflow_run_id)
    return wes_get_request(wes, target)

In [22]:
get_workflow_run_status(wes, list_workflow_runs(wes)['workflows'][-1]['workflow_id'])

{u'state': u'Error', u'workflow_id': u'cc9e9abf668342658b893ebea793576d'}

## Submitting workflows

Use `build_workflow_run_packet` to collect and format parameters/instructions for a new workflow run.

In [31]:
def build_workflow_run_packet(
    workflow_params, workflow_type, workflow_type_version, workflow_descriptor=None, 
    tags=None, workflow_engine_parameters=None, workflow_url=None, key_values=None
):
    """
    Build JSON request packet for submitting a workflow run.
    
    Fields from http://ga4gh.github.io/workflow-execution-service-schemas/definitions/ga4gh_wes_WorkflowRequest
    """
    packet = {
        'workflow_descriptor': workflow_descriptor, 
        'workflow_params' : workflow_params, 
        'workflow_type': workflow_type,
        'workflow_type_version': workflow_type_version,
        'tags': tags,
        'workflow_engine_parameters': workflow_engine_parameters,
        'workflow_url': workflow_url,
        'key_values' : key_values
    }
    return {k: v for k, v in packet.items()
            if v is not None}
    

`read_file` is a small helper function from Abraham (used here to read in JSON parameters file).

In [32]:
def read_file(file_name):
    file_r = open(file_name,'r')
    return file_r.read()

### 1. Testing with `dockstore-tool-bamstats`

Example workflow from Abraham's gist/notebook. Check out his notebook for the contents of `bams.json` (I just copied and pasted into a local file).

In [61]:
# File in local path
params = json.dumps(json.loads(read_file('bams.json')))

workflow_type = 'cwl'
workflow_type_version = '1.0'
key_values = {'flavour' : 'c4.xlarge'} # Consonance specific key value

Create a generic `POST` request function (to minimize repeated code). Returns the response of the request.

In [66]:
def wes_post_request(wes, target, json):
    """
    Generic workflow execution service API 'POST' request.
    """
    res = requests.post(
        '{}{}'.format(wes.endpoint, target),
        headers=wes.headers,
        json=json
    )
    # TODO: add some exception handling for different responses
    return res.json()

Use `submit_workflow_run` to initiate a new workflow run with the specified parameters. The response for this request will give the ID of the submitted workflow run.

In [67]:
def submit_workflow_run(wes, workflow_run_packet, tags=None):
    target = '/workflows'
    params = json.loads(workflow_run_packet['workflow_params'])
#     params.update(tags)
    workflow_run_packet['workflow_params'] = params
    # print request data for testing
    print(workflow_run_packet)
    return wes_post_request(wes, target, json=workflow_run_packet)

#### 1.1. Using Dockstore ID

This test follows Abraham's example, using a Dockstore tool ID for the `workflow_descriptor` field. 

> I don't think this technically conforms with the WES spec (?) — my understanding is that `_descriptor` should be the actual descriptor object, and `_url` should be a URL path to the descriptor.

In [36]:
# Dockstore tool id.
descriptor = "quay.io/collaboratory/dockstore-tool-bamstats:1.25-6_1.0" 

In [69]:
test1_1 = submit_workflow_run(
    wes, 
    workflow_run_packet = build_workflow_run_packet(
        workflow_descriptor=descriptor,
        workflow_params=params,
        workflow_type=workflow_type,
        workflow_type_version=workflow_type_version,
        key_values=key_values
    )
)
test1_1

{'key_values': {'flavour': 'c4.xlarge'}, 'workflow_type_version': '1.0', 'workflow_params': {u'bam_input': {u'path': u'https://s3-us-west-2.amazonaws.com/achave11-redwood-storage/consonance-outputs/rna.SRR948778.bam', u'class': u'File', u'format': u'http://edamontology.org/format_2572'}, u'bamstats_report': {u'path': u'/tmp/bamstats_report.zip', u'class': u'File'}}, 'workflow_descriptor': 'quay.io/collaboratory/dockstore-tool-bamstats:1.25-6_1.0', 'workflow_type': 'cwl'}


ValueError: No JSON object could be decoded

> So, that didn't seem to work... 

In [38]:
list_workflow_runs(wes)

{u'next_page_token': u'',
 u'workflows': [{u'state': u'Complete',
   u'workflow_id': u'6ab6a9e2728b4068bca19855071a2155'},
  {u'state': u'Error', u'workflow_id': u'8e398fb0b4d043a2a3b534c1b063fd57'},
  {u'state': u'Running', u'workflow_id': u'7efd9cde924345768423225faf934d58'},
  {u'state': u'Error', u'workflow_id': u'8db78ed7cd324912a8af65d83f1716b6'},
  {u'state': u'Complete', u'workflow_id': u'9002c8e40f67404c9ffbeab5f6428992'},
  {u'state': u'Error', u'workflow_id': u'cc9e9abf668342658b893ebea793576d'}]}

Add a little function to find the actual ID of the workflow run.

In [23]:
def find_workflow_run(wes, tag_key, tag_val, n=5):
    wr_list = []
    for wr in list_workflow_runs(wes)['workflows'][-n:]:
        params = json.loads(
            get_workflow_run(wes, wr['workflow_id'])['request']['workflow_params']
        )
        wr_tag_val = params.get(tag_key, None)
        if wr_tag_val == tag_val:
            wr_list.append(wr['workflow_id'])
    return wr_list[-1]

In [24]:
test1_1_id = find_workflow_run(wes, 'track_id', 'test1_1')
test1_1_id

u'47'

OK, so the now that I have the actual workflow run ID, I can check status and details using the same functions as above.

In [27]:
get_workflow_run_status(wes, test1_1_id)

{u'state': u'Complete', u'workflow_id': u'47'}

> Assume I re-ran the above cell until the `state` was `Complete`.

In [28]:
get_workflow_run(wes, test1_1_id)

{u'outputs': [{u'location': None,
   u'name': None,
   u'type': None,
   u'value': None}],
 u'request': {u'key_values': None,
  u'workflow_descriptor': u'#!/usr/bin/env cwl-runner\n\nclass: CommandLineTool\nid: "BAMStats"\nlabel: "BAMStats tool"\ncwlVersion: v1.0 \ndescription: |\n    ![build_status](https://quay.io/repository/collaboratory/dockstore-tool-bamstats/status)\n    A Docker container for the BAMStats command. See the [BAMStats](http://bamstats.sourceforge.net/) website for more information.\n\ndct:creator:\n  "@id": "http://orcid.org/0000-0002-7681-6415"\n  foaf:name: Brian O\'Connor\n  foaf:mbox: "mailto:briandoconnor@gmail.com"\n\nrequirements:\n  - class: DockerRequirement\n    dockerPull: "quay.io/collaboratory/dockstore-tool-bamstats:1.25-6_1.0"\n\nhints:\n  - class: ResourceRequirement\n    coresMin: 1\n    ramMin: 4092\n    outdirMin: 512000\n    doc: "the process requires at least 4G of RAM"\n\ninputs:\n  mem_gb:\n    type: int\n    default: 4\n    doc: "The memory,

#### Test 1.1 Results

> Aside from the fact that my `key_values` don't appear to have been correctly parsed/set from the request, everything else seems to have worked fine.

#### Using CWL content

Next attempt is to use the actual CWL content/document for the `workflow_descriptor` field. First, I'll grab the content from the descriptor URL.

In [70]:
# Dockstore tool URL
descriptor_url = 'https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fcollaboratory%2Fdockstore-tool-bamstats/versions/1.25-6_1.0/plain-CWL/descriptor/%2FDockstore.cwl'

# Get descriptor from URL
descriptor_content = requests.get(descriptor_url).content

Descriptor content looks like:

In [71]:
descriptor_content

'#!/usr/bin/env cwl-runner\n\nclass: CommandLineTool\nid: "BAMStats"\nlabel: "BAMStats tool"\ncwlVersion: v1.0 \ndescription: |\n    ![build_status](https://quay.io/repository/collaboratory/dockstore-tool-bamstats/status)\n    A Docker container for the BAMStats command. See the [BAMStats](http://bamstats.sourceforge.net/) website for more information.\n\ndct:creator:\n  "@id": "http://orcid.org/0000-0002-7681-6415"\n  foaf:name: Brian O\'Connor\n  foaf:mbox: "mailto:briandoconnor@gmail.com"\n\nrequirements:\n  - class: DockerRequirement\n    dockerPull: "quay.io/collaboratory/dockstore-tool-bamstats:1.25-6_1.0"\n\nhints:\n  - class: ResourceRequirement\n    coresMin: 1\n    ramMin: 4092\n    outdirMin: 512000\n    doc: "the process requires at least 4G of RAM"\n\ninputs:\n  mem_gb:\n    type: int\n    default: 4\n    doc: "The memory, in GB, for the reporting tool"\n    inputBinding:\n      position: 1\n\n  bam_input:\n    type: File\n    doc: "The BAM file used as input, it must be s

In [72]:
test1_2 = submit_workflow_run(
    wes, 
    build_workflow_run_packet(
        workflow_descriptor=descriptor_content,
        workflow_params=params,
        workflow_type=workflow_type,
        workflow_type_version=workflow_type_version,
        key_values=key_values
    )
)
test1_2

{'key_values': {'flavour': 'c4.xlarge'}, 'workflow_type_version': '1.0', 'workflow_params': {u'bam_input': {u'path': u'https://s3-us-west-2.amazonaws.com/achave11-redwood-storage/consonance-outputs/rna.SRR948778.bam', u'class': u'File', u'format': u'http://edamontology.org/format_2572'}, u'bamstats_report': {u'path': u'/tmp/bamstats_report.zip', u'class': u'File'}}, 'workflow_descriptor': '#!/usr/bin/env cwl-runner\n\nclass: CommandLineTool\nid: "BAMStats"\nlabel: "BAMStats tool"\ncwlVersion: v1.0 \ndescription: |\n    ![build_status](https://quay.io/repository/collaboratory/dockstore-tool-bamstats/status)\n    A Docker container for the BAMStats command. See the [BAMStats](http://bamstats.sourceforge.net/) website for more information.\n\ndct:creator:\n  "@id": "http://orcid.org/0000-0002-7681-6415"\n  foaf:name: Brian O\'Connor\n  foaf:mbox: "mailto:briandoconnor@gmail.com"\n\nrequirements:\n  - class: DockerRequirement\n    dockerPull: "quay.io/collaboratory/dockstore-tool-bamstats:

ValueError: No JSON object could be decoded

In [32]:
test1_2_id = find_workflow_run(wes, 'track_id', 'test1_2')
test1_2_id

u'48'

In [41]:
get_workflow_run_status(wes, test1_2_id)

{u'state': u'Complete', u'workflow_id': u'48'}

In [42]:
get_workflow_run(wes, test1_2_id)

{u'outputs': [{u'location': None,
   u'name': None,
   u'type': None,
   u'value': None}],
 u'request': {u'key_values': None,
  u'workflow_descriptor': None,
  u'workflow_params': u'{"bam_input": {"path": "https://s3-us-west-2.amazonaws.com/achave11-redwood-storage/consonance-outputs/rna.SRR948778.bam", "class": "File", "format": "http://edamontology.org/format_2572"}, "bamstats_report": {"path": "/tmp/bamstats_report.zip", "class": "File"}, "track_id": "test1_2"}',
  u'workflow_type': u'cwl',
  u'workflow_type_version': u'1.0'},
 u'state': u'Complete',
 u'task_logs': [{u'cmd': None,
   u'endTime': u'2018-05-03 04:22:01.895187',
   u'exitCode': None,
   u'name': u'cwl',
   u'startTime': u'2018-05-03 04:20:07.436',
   u'stderr': u"/usr/local/bin/cwltool 1.0.20160712154127\n['docker', 'pull', 'quay.io/collaboratory/dockstore-tool-bamstats:1.25-6_1.0']\n1.25-6_1.0: Pulling from collaboratory/dockstore-tool-bamstats\n3f992ab3df53: Pulling fs layer\n0aa0bd28396f: Pulling fs layer\ndb7bb1508

#### Test 1.2 Results

> `workflow_descriptor` appears to be unset (failed to parse?); however, there's no other indication that the job failed. I assume that the `stderr` is held over from a previous job? Or maybe not. Hard to tell what's happening...

#### 1.3 Using descriptor URL

Now I'll provide the descriptor URL in the `workflow_url` field — which, based on my understanding of the WES spec, is a valid option.

In [43]:
test1_3 = submit_workflow_run(
    wes, 
    build_workflow_run_packet(
        workflow_url=descriptor_url,
        workflow_params=params,
        workflow_type=workflow_type,
        workflow_type_version=workflow_type_version,
        key_values=key_values
    ),
    tags={'track_id': 'test1_3'}
)
test1_3

{'key_values': {'flavour': 'c4.xlarge', 'tracker_id': 'test1'}, 'workflow_url': 'https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fcollaboratory%2Fdockstore-tool-bamstats/versions/1.25-6_1.0/plain-CWL/descriptor/%2FDockstore.cwl', 'workflow_type_version': '1.0', 'workflow_params': '{"bam_input": {"path": "https://s3-us-west-2.amazonaws.com/achave11-redwood-storage/consonance-outputs/rna.SRR948778.bam", "class": "File", "format": "http://edamontology.org/format_2572"}, "bamstats_report": {"path": "/tmp/bamstats_report.zip", "class": "File"}, "track_id": "test1_3"}', 'workflow_type': 'cwl'}


{u'code': 400, u'message': u'Unable to process JSON'}

#### Test1.3 Results

> Fail. :(

#### 1.4 Using descriptor URL, part 2

Maybe if I provide the descriptor URL as the `workflow_descriptor`...

In [44]:
test1_4 = submit_workflow_run(
    wes, 
    build_workflow_run_packet(
        workflow_descriptor=descriptor_url,
        workflow_params=params,
        workflow_type=workflow_type,
        workflow_type_version=workflow_type_version,
        key_values=key_values
    ),
    tags={'track_id': 'test1_4'}
)
test1_4

{'key_values': {'flavour': 'c4.xlarge', 'tracker_id': 'test1'}, 'workflow_type_version': '1.0', 'workflow_params': '{"bam_input": {"path": "https://s3-us-west-2.amazonaws.com/achave11-redwood-storage/consonance-outputs/rna.SRR948778.bam", "class": "File", "format": "http://edamontology.org/format_2572"}, "bamstats_report": {"path": "/tmp/bamstats_report.zip", "class": "File"}, "track_id": "test1_4"}', 'workflow_descriptor': 'https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fcollaboratory%2Fdockstore-tool-bamstats/versions/1.25-6_1.0/plain-CWL/descriptor/%2FDockstore.cwl', 'workflow_type': 'cwl'}


{u'workflow_id': u'33'}

In [45]:
test1_4_id = find_workflow_run(wes, 'track_id', 'test1_4')
test1_4_id

u'49'

In [47]:
get_workflow_run_status(wes, test1_4_id)

{u'state': u'Complete', u'workflow_id': u'49'}

In [48]:
get_workflow_run(wes, test1_4_id)

{u'outputs': [{u'location': None,
   u'name': None,
   u'type': None,
   u'value': None}],
 u'request': {u'key_values': None,
  u'workflow_descriptor': u'#!/usr/bin/env cwl-runner\n\nclass: CommandLineTool\nid: "BAMStats"\nlabel: "BAMStats tool"\ncwlVersion: v1.0 \ndescription: |\n    ![build_status](https://quay.io/repository/collaboratory/dockstore-tool-bamstats/status)\n    A Docker container for the BAMStats command. See the [BAMStats](http://bamstats.sourceforge.net/) website for more information.\n\ndct:creator:\n  "@id": "http://orcid.org/0000-0002-7681-6415"\n  foaf:name: Brian O\'Connor\n  foaf:mbox: "mailto:briandoconnor@gmail.com"\n\nrequirements:\n  - class: DockerRequirement\n    dockerPull: "quay.io/collaboratory/dockstore-tool-bamstats:1.25-6_1.0"\n\nhints:\n  - class: ResourceRequirement\n    coresMin: 1\n    ramMin: 4092\n    outdirMin: 512000\n    doc: "the process requires at least 4G of RAM"\n\ninputs:\n  mem_gb:\n    type: int\n    default: 4\n    doc: "The memory,

#### Test 1.4 Results

> Inconclusive. The descriptor document seems to have been correctly retrieved and set, but I'm still unsure if I can trust the `stderr`. I probably need to test sequentially with different workflows.